In [ ]:
# 참고 : https://github.com/kairess/colorizer

In [ ]:
# 내 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import os
import numpy as np
#import matplotlib.pyplot as plt
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('/content/gdrive/MyDrive/CV2/colorize/Youngman.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
file_name = '/content/gdrive/MyDrive/CV2/colorize/Youngman.mp4'  # 원본 동영상
min_confidence = 0.5   # detection 으로 인정할 최소 확률(신뢰도) 지정
output_name = 'Output_video.mp4'  # output 동영상 이름
elapsed_time = 0       # 총 경과시간 초기화 

In [ ]:
def detectAndDisplay(frame):
  start_time = time.time()
    
  #img = cv2.resize(img, output_size)

  pred_bgr = frame.copy()
  img_ori = frame.copy()

  # normalize input
  #img_ori = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
  #img_ori = cv2.cvtColor(img_ori, cv2.COLOR_GRAY2RGB)
  img_ori = cv2.cvtColor(img_ori, cv2.COLOR_BGR2RGB)

  img_ori = (img_ori / 255.).astype(np.float32)

  # convert RGB to LAB
  img_lab = cv2.cvtColor(img_ori, cv2.COLOR_RGB2Lab)
  # only L channel to be used
  img_l = img_lab[:, :, 0]

  input_img = cv2.resize(img_l, (224, 224))
  input_img -= 50 # subtract 50 for mean-centering

  # prediction
  net.setInput(cv2.dnn.blobFromImage(input_img))

  pred = net.forward()[0,:,:,:].transpose((1, 2, 0))

  # resize to original image shape
  pred_resize = cv2.resize(pred, (img_ori.shape[1], img_ori.shape[0]))

  # concatenate with original image L
  pred_lab = np.concatenate([img_l[:, :, np.newaxis], pred_resize], axis=2)

  # convert LAB to RGB
  pred_bgr = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2BGR)
  pred_bgr = np.clip(pred_bgr, 0, 1) * 255
  pred_bgr = pred_bgr.astype(np.uint8)

  process_time = time.time() - start_time
  global elapsed_time
  elapsed_time += process_time   # 총 경과시간 누적
  print("=== A frame took {:.3f} seconds".format(process_time))

  # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
  global writer
  if writer is None and output_name is not None:
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter(output_name, fourcc, 30,
              (img_ori.shape[1], img_ori.shape[0]), True)
      
  # disk 에 frame 을 write 합니다.
  if writer is not None:
      writer.write(pred_bgr)

In [ ]:
proto = '/content/gdrive/MyDrive/CV2/colorize/models/colorization_deploy_v2.prototxt'
#weights = '/content/gdrive/MyDrive/CV2/colorize/models/colorization_release_v2.caffemodel'
weights = '/content/gdrive/MyDrive/CV2/colorize/models/colorization_release_v2_norebal.caffemodel'
# load cluster centers
pts_in_hull = np.load('/content/gdrive/MyDrive/CV2/colorize/models/pts_in_hull.npy')
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1).astype(np.float32)

# load model
net = cv2.dnn.readNetFromCaffe(proto, weights)
# net.getLayerNames()

# populate cluster centers as 1x1 convolution kernel
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full((1, 313), 2.606, np.float32)]

# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)

=== A frame took 0.988 seconds
=== A frame took 0.980 seconds
=== A frame took 0.975 seconds
=== A frame took 0.981 seconds
=== A frame took 0.976 seconds
=== A frame took 0.978 seconds
=== A frame took 0.970 seconds
=== A frame took 1.002 seconds
=== A frame took 0.967 seconds
=== A frame took 0.974 seconds
=== A frame took 0.972 seconds
=== A frame took 0.971 seconds
=== A frame took 1.009 seconds
=== A frame took 0.974 seconds
=== A frame took 0.970 seconds
=== A frame took 0.975 seconds
=== A frame took 0.980 seconds
=== A frame took 0.972 seconds
=== A frame took 0.971 seconds
=== A frame took 0.979 seconds
=== A frame took 0.961 seconds
=== A frame took 0.969 seconds
=== A frame took 0.967 seconds
=== A frame took 0.974 seconds
=== A frame took 0.970 seconds
=== A frame took 0.970 seconds
=== A frame took 0.964 seconds
=== A frame took 0.970 seconds
=== A frame took 0.974 seconds
=== A frame took 0.979 seconds
=== A frame took 0.975 seconds
=== A frame took 0.976 seconds
=== A fr